In [1]:
from IPython.display import HTML
import pandas as pd
import altair as alt

In [2]:
data_pdf = pd.DataFrame({
    'progression_level' : [1, 2, 3, 4, 5, 6, 7],
    'progression_label': ['AU', 'GAU', '≥2 Play Seqs.', '≥ Pro 2 Division', '≥1 Demoted', '≥1 Add. Promo Supp.', ''],
    'user_count': [2258930, 1110052, 530386, 388562, 156791, 15413, 15413],
    'user_count_pct': ['100%', '49.1%', '23.5%', '17.2%', '6.94%', '0.68%', ''],
    'user_count_aps_pct': ['0.68%', '1.38%', '2.91%', '3.97%', '9.83%', '100%', ''],
})


In [3]:
progression_level_picker = alt.selection_point(fields=['progression_level'], on='mouseover', empty=True)
progression_level_picker_for_aps_pct = alt.selection_point(fields=['progression_level'], on='mouseover', empty=False)

In [4]:
base = alt.Chart(data_pdf)

In [5]:
area_ap = (
    base
        .transform_calculate(
            user_count_y = 'datum.user_count/2',
            user_count_y2 = '-datum.user_count/2'
        )
        .encode(
            x=alt.X('progression_level:O')
                .axis(
                    grid=True,
                    ticks=False,
                    domain=False,
                    labels=False,
                    title=None,
                    zindex=1
                ),
            y=alt.Y('user_count_y:Q')
                .axis(None),
            y2=alt.Y2('user_count_y2:Q'),
            color=alt.value('#FF6A14'),
        )
        .mark_area(interpolate='monotone')
)

In [6]:
user_count_text_ap = (
    alt.Chart(data_pdf[:-1])
        .transform_calculate(
            user_count_y = 'datum.user_count/2'
        )
        .transform_joinaggregate(
            text_y_raw='max(user_count_y)'
        )
        .transform_calculate(
            text_y='datum.text_y_raw * 1.8'
        )
        .encode(
            y=alt.Y('text_y:Q'),
            x=alt.X('progression_level:O'),
            text=alt.Text('user_count:Q')
                .format(','),
            color=alt.value('black')
        )
        .mark_text(
            align='left',
            baseline='top',
            dx=10,
            dy=-25,
            fontSize=14,
            fontWeight=500            
        )
)

In [7]:
progression_label_text_ap = (
    user_count_text_ap
        .encode(
            y=alt.Y('text_y:Q'),
            text=alt.Text('progression_label:N'),
            color=alt.value('black')
        )
        .mark_text(
            align='left',
            baseline='top',
            dx=10,
            dy=-5
            #dy=-10,
            #fontSize=12,
            #fontWeight=700
        )
)

In [8]:
user_count_pct_text_ap = (
    user_count_text_ap
        .encode(
            y=alt.Y('text_y:Q'),
            text=alt.Text('user_count_pct:N'),
            color=alt.value('black')
        )
        .mark_text(
            align='left',
            baseline='top',
            dx=10,
            dy=10
        )
)

In [9]:
progression_cover_bar_ap = (
    user_count_text_ap
        .transform_calculate(
            cover_bar_y='datum.text_y_raw*2.2',
            cover_bar_y2='-datum.text_y_raw*1.2'
        )
        .encode(
            y=alt.Y('cover_bar_y:Q'),
            y2=alt.Y2('cover_bar_y2:Q'),
            color=alt.value('white'),
            opacity=alt.condition(progression_level_picker, alt.value(0), alt.value(0.8))
        )
        .mark_bar(opacity=0.5, xOffset=57)
        .add_params(progression_level_picker)
        .add_params(progression_level_picker_for_aps_pct)
)

In [10]:
user_count_aps_pct_text_ap = (
    user_count_text_ap
        .transform_calculate(
            aps_pct_x='6',
            aps_pct_y='0',
            user_count_aps_pct_formatted="datum.user_count_aps_pct+' of'"
        )
        .encode(
            x=alt.X('aps_pct_x:O'),
            y=alt.Y('aps_pct_y:Q'),
            text=alt.Text('user_count_aps_pct_formatted:N'),
            opacity=alt.condition(progression_level_picker_for_aps_pct, alt.value(0.8), alt.value(0))
        )
        .mark_text(yOffset=-30, xOffset=57)
)


user_count_aps_pct_support_text_ap = (
    user_count_aps_pct_text_ap
        .encode(
            text=alt.Text('progression_label:N'),
        )
        .mark_text(yOffset=-15, xOffset=57)
)

user_count_aps_pct_arrow_text_ap = (
    user_count_aps_pct_text_ap
        .transform_calculate(
            arrow="'↓'"
        )
        .encode(
            text=alt.Text('arrow:N'),
        )
        .mark_text(yOffset=-3, xOffset=57)    
)

aps_area_ap = (
    alt.Chart(data_pdf[-2:])
        .transform_calculate(
            user_count_y = 'datum.user_count/2*0.8',
            user_count_y2 = '-datum.user_count/2*0.8'
        )
        .encode(
            x=alt.X('progression_level:O'),
            y=alt.Y('user_count_y:Q'),
            y2=alt.Y2('user_count_y2:Q'),
            color=alt.value('#E80005'),
        )
        .mark_area(interpolate='monotone')
)


In [11]:
main_funnel_ap = (
    alt.layer(
        area_ap,
        progression_label_text_ap,
        user_count_text_ap,
        user_count_pct_text_ap
    )
)

In [12]:
HTML("<h4>Targeted Users</h4>")

In [13]:
HTML("<p>Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua.<br> Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. </p>")

In [14]:
HTML("<br>")

In [15]:
final_ap = (
    alt.layer(
        main_funnel_ap,
        progression_cover_bar_ap,
        user_count_aps_pct_text_ap,
        user_count_aps_pct_support_text_ap,
        user_count_aps_pct_arrow_text_ap,
        aps_area_ap
    )
    .properties(
        width=800,
        height=250
    )
    .configure_view(
        stroke = None
    )

)
final_ap

alt.LayerChart(...)